In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor  # Import MLPRegressor
from sklearn.metrics import mean_squared_error
import csv
from sklearn.preprocessing import RobustScaler, LabelEncoder


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the training data
df = pd.read_csv('train.csv')
df.drop_duplicates(inplace=True)
df = df.drop(columns=['sub_area'], axis=1)

# Separate features (X) and target variable (y)
X = df.loc[:, df.columns != 'price_doc']
y = df['price_doc']

# Encode categorical variables
object_cols = X.select_dtypes(include=['object']).columns
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

# Apply feature scaling
scaler = RobustScaler()
X = scaler.fit_transform(X)

# Apply PCA
PCA_s = PCA(n_components=15)
X = PCA_s.fit_transform(X)

# Apply variance threshold
var = VarianceThreshold(threshold=0.01)
X = var.fit_transform(X)

# Polynomial features
poly = PolynomialFeatures(3, include_bias=False)
X = poly.fit_transform(X)

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11272\771146571.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11272\771146571.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11272\771146571.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_test = model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error on Test Data: {mse}')

# Load the test data
df_test = pd.read_csv('test.csv')
df_test = df_test.drop(['row ID'], axis=1)
df_test = df_test.drop(['sub_area'], axis=1)

# Encode categorical variables in the test data
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))
# Apply feature scaling, PCA, variance threshold, feature selection, and polynomial features to the test data
df_test = scaler.transform(df_test)
df_test = PCA_s.transform(df_test)
df_test = var.transform(df_test)
df_test = poly.transform(df_test)

Mean Squared Error on Test Data: 177859734625839.2


In [ ]:
# Apply neural network
nn_model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=100)  # You can adjust the parameters as needed
nn_model.fit(X_train, y_train)

# Make predictions on the test data using the trained neural network
y_pred_test_final = nn_model.predict(df_test)


c:\Users\Irtiza\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Save predictions to a CSV file
filepath = 'prediction_26nov_nn.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Add column headers
    writer.writerow(['row ID', 'price_doc'])

    # Writing predictions with corresponding index (starting from 1)
    for c, prediction in enumerate(y_pred_test_final, start=1):
        writer.writerow([c, prediction])